In [1]:
import mediapipe as mp 
import cv2

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [53]:
camera = cv2.VideoCapture(1)
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands = 1) as hands:
    while camera.isOpened():
        _, image = camera.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 
        results = hands.process(image)

        image.flags.writeable = True
        landmark_list = []
        if results.multi_hand_landmarks:
            for landmark in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, landmark, mp_hands.HAND_CONNECTIONS)
            
        cv2.imshow("Camera No. 1", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        if cv2.waitKey(10) & 0xff == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()

In [30]:
num_coord = len(results.multi_hand_landmarks[0].landmark)
num_coord

21

In [82]:
import csv
import numpy as np 
import os

In [73]:
landmarks = ['class']
for num in range(1, num_coord + 1):
    landmarks += [f'x{num}', f'y{num}', f'z{num}'] 

In [48]:
landmarks
len(landmarks)

85

In [74]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)
    

In [49]:
class_id = "thumbs up"

In [84]:
camera = cv2.VideoCapture(1)
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands = 1) as hands:
    while camera.isOpened():
        _, image = camera.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 
        results = hands.process(image)

        image.flags.writeable = True
        landmark_list = []
        if results.multi_hand_landmarks:
            for landmark in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, landmark, mp_hands.HAND_CONNECTIONS)
        
        try:
            hand = enumerate(results.multi_hand_landmarks[0].landmark)
            hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for _, landmark in hand]).flatten())
            hand_row.insert(0, class_id)
            print(hand_row)
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(hand_row)
                
        except Exception as e:
            pass
            
        cv2.imshow("Camera No. 1", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        if cv2.waitKey(10) & 0xff == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()

['thumbs up', 0.9000928997993469, 0.7851548790931702, 1.4565198398486245e-05, 0.8645116090774536, 0.6434077024459839, 0.016184313222765923, 0.8058549761772156, 0.5225313901901245, 0.03684181347489357, 0.773894190788269, 0.4360198676586151, 0.055705782026052475, 0.7603024840354919, 0.36744925379753113, 0.06699040532112122, 0.7611123323440552, 0.5851973295211792, 0.0489286333322525, 0.770383894443512, 0.5930713415145874, 0.0881938636302948, 0.7951814532279968, 0.6081036925315857, 0.08620511740446091, 0.8153418898582458, 0.6137827038764954, 0.06948988139629364, 0.7771797776222229, 0.6433651447296143, 0.04842172935605049, 0.7831993699073792, 0.6430463194847107, 0.1184156984090805, 0.8074656128883362, 0.6652839183807373, 0.10851441323757172, 0.8269964456558228, 0.6753835082054138, 0.07397382706403732, 0.7983078360557556, 0.6979354023933411, 0.05900824815034866, 0.8064761757850647, 0.6903634071350098, 0.11141809076070786, 0.8296514749526978, 0.7235099077224731, 0.09222649037837982, 0.8442432

['thumbs up', 0.788997232913971, 0.5888362526893616, -0.00016910773410927504, 0.7539946436882019, 0.46116742491722107, -0.026147078722715378, 0.7431801557540894, 0.32749253511428833, -0.07544118911027908, 0.7381371259689331, 0.22176724672317505, -0.112258680164814, 0.7285090088844299, 0.12296858429908752, -0.13858391344547272, 0.8240658044815063, 0.2963305711746216, -0.16893011331558228, 0.7259520888328552, 0.32792001962661743, -0.21345175802707672, 0.7086944580078125, 0.3792613744735718, -0.19481942057609558, 0.7229725122451782, 0.38575461506843567, -0.18814866244792938, 0.8499834537506104, 0.38637515902519226, -0.1872502565383911, 0.7312245965003967, 0.43129557371139526, -0.23176662623882294, 0.7239182591438293, 0.46547263860702515, -0.1730354279279709, 0.7467908263206482, 0.4570518136024475, -0.1395650953054428, 0.8584776520729065, 0.48471277952194214, -0.21036115288734436, 0.7446233630180359, 0.5276957154273987, -0.23391203582286835, 0.7426327466964722, 0.5480971336364746, -0.17423

['thumbs up', 0.7928764224052429, 0.6282183527946472, -0.00012707957648672163, 0.7427202463150024, 0.4960925579071045, -0.003817650256678462, 0.7339529991149902, 0.3499454855918884, -0.039646901190280914, 0.7267934083938599, 0.23217816650867462, -0.06821684539318085, 0.7094424366950989, 0.12515667080879211, -0.09125900268554688, 0.8277763724327087, 0.29369500279426575, -0.1188315898180008, 0.7281732559204102, 0.3308960795402527, -0.13503722846508026, 0.7120503783226013, 0.39022865891456604, -0.10632594674825668, 0.7294893264770508, 0.39794114232063293, -0.10454629361629486, 0.8534450531005859, 0.3694140911102295, -0.14362166821956635, 0.7346450090408325, 0.40556392073631287, -0.17251230776309967, 0.729182243347168, 0.4506050646305084, -0.13468722999095917, 0.7532610297203064, 0.4543541967868805, -0.1184016540646553, 0.8610800504684448, 0.4568975567817688, -0.16645127534866333, 0.7467003464698792, 0.4922483563423157, -0.18038830161094666, 0.7446542978286743, 0.5310149192810059, -0.13841